In [35]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

city="Houston"
state="TX"
county="HARRIS COUNTY"
zipcode="7002"


In [36]:
# ---------------------------------- DATA IMPORT ----------------------------------
# Sourcing data from https://www.zillow.com/research/data/

In [37]:
# Create a function that will transpose multiple date columns into Data vs Value
def melt_zillow_data(data, var_name):

    # identify columns not containing date reference
    id_vars=data.columns[~data.columns.str.contains('-')]

    # set up tanspose logic
    melted_data=pd.melt(data,
                       id_vars=id_vars,
                       var_name="Date",
                       value_name=var_name)
    # return transposed df   
    return melted_data

In [39]:
# Creating a function to pull data from Zillow (in *.csv format)

def pull_zillow_data(url, state, county, city, var_name):
#   Read in data from Zillow live url reference   
    data=pd.read_csv(url) # direct url
    
    data=data.drop(['RegionID', 'SizeRank'],axis=1)
    data=melt_zillow_data(data,var_name)
    
    
    return(data)
   

In [42]:
# define a rangle of string variables with url links to live *.csv format data
# listing price
list_price_data="https://files.zillowstatic.com/research/public_v2/mlp/Metro_mlp_uc_sfrcondo_raw_month.csv"
# sale price
sale_data="https://files.zillowstatic.com/research/public_v2/median_sale_price/Metro_median_sale_price_uc_SFRCondo_raw_month.csv"

# inventory data (days to pending)
inventory_d2p_data="http://files.zillowstatic.com/research/public_v2/mean_doz_pending/Metro_mean_doz_pending_uc_sfrcondo_raw_monthly.csv"
# for sale inventory count
inventory_4s_data="https://files.zillowstatic.com/research/public_v2/invt_fs/Metro_invt_fs_uc_sfrcondo_raw_month.csv"

# ----------------------------------- DATA BY ZIP CODE -----------------------------------
zhvi_single_family_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_condo_coop_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_uc_condo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_5plus_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_bdrmcnt_5_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_4_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_3_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_2_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_1_homes_by_zip="https://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"

# ----------------------------------- DATA BY COUNTY -----------------------------------
zhvi_single_family_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_condo_coop_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_uc_condo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_5plus_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_bdrmcnt_5_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_4_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_3_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_2_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zhvi_1_homes_by_county="https://files.zillowstatic.com/research/public_v2/zhvi/County_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"


In [73]:
url_dict=[
# Data by CITY    
    {
        "stat_name"   : "list_price_data",
        "url"         : list_price_data,
        "description" : f"Median List Price: The median price at which homes across various geographies were listed.\nRaw. All homes. Monthly view."
    },
    {   "stat_name"   : "sale_data",
        "url"         : sale_data,
        "description" : f"Median Sale Price: The median price at which homes across various geographies were sold.\nRaw. All homes. Monthly view."
    },
    {   "stat_name"   : "inventory_d2p_data",
        "url"         : inventory_d2p_data,
        "description" : f"Days to Pending: How long it takes homes in a region to change to pending status on Zillow.com after first being shown as for sale."
    },
    {   "stat_name"   : "inventory_4s_data",
        "url"         : inventory_4s_data,
        "description" : f"For-Sale Inventory: The count of unique listings that were active at any time in a given month."
    },
# Data by ZIP CODE    
    {   "stat_name"   : "zhvi_single_family_homes_by_zip",
        "url"         : zhvi_single_family_homes_by_zip,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\nAll single family time series by Zipcode"
    },
    {   "stat_name"   : "zhvi_condo_coop_homes_by_zip",
        "url"         : zhvi_condo_coop_homes_by_zip,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\nAll condo/co-op time series by Zipcode"
    },
    {   "stat_name"   : "zhvi_5plus_homes_by_zip",
        "url"         : zhvi_5plus_data,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n5+ bed time series by Zipcode"
    },
    {   "stat_name"   : "zhvi_4_homes_by_zip",
        "url"         : zhvi_4_homes_by_zip,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n4 bed time series by Zipcode"
    },
    {   "stat_name"   : "zhvi_3_homes_by_zip",
        "url"         : zhvi_3_homes_by_zip,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n3 bed time series by Zipcode"
    },
    {   "stat_name"   : "zhvi_2_homes_by_zip",
        "url"         : zhvi_2_homes_by_zip,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n2 bed time series by Zipcode"
    },
    {   "stat_name"   : "zhvi_1_homes_by_zip",
        "url"         : zhvi_1_homes_by_zip,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n1 bed time series by Zipcode"
    },
# Data by COUNTY        
    {   "stat_name"   : "zhvi_single_family_homes_by_county",
        "url"         : zhvi_single_family_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\nAll single family time series by county"
    },
    {   "stat_name"   : "zhvi_condo_coop_homes_by_county",
        "url"         : zhvi_condo_coop_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\nAll condo/co-op time series by county"
    },
    {   "stat_name"   : "zhvi_5plus_homes_by_county",
        "url"         : zhvi_5plus_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n5+ bed time series by county"
    },
    {   "stat_name"   : "zhvi_4_homes_by_county",
        "url"         : zhvi_4_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n4 bed time series by county"
    },
    {   "stat_name"   : "zhvi_3_homes_by_county",
        "url"         : zhvi_3_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n3 bed time series by county"
    },
    {   "stat_name"   : "zhvi_2_homes_by_county",
        "url"         : zhvi_2_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n2 bed time series by county"
    },
    {   "stat_name"   : "zhvi_1_homes_by_county",
        "url"         : zhvi_1_homes_by_county,
        "description" : f"Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes.\n1 bed time series by county"}

]


zillow_df=pd.DataFrame(url_dict)
zillow_df['data']=""
zillow_df['type']=""
zillow_df

,stat_name,url,description,data,type
0,list_price_data,https://files.zillowstatic.com/research/public...,Median List Price: The median price at which h...,,
1,sale_data,https://files.zillowstatic.com/research/public...,Median Sale Price: The median price at which h...,,
2,inventory_d2p_data,http://files.zillowstatic.com/research/public_...,Days to Pending: How long it takes homes in a ...,,
3,inventory_4s_data,https://files.zillowstatic.com/research/public...,For-Sale Inventory: The count of unique listin...,,
4,zhvi_5plus_homes_by_zip,https://files.zillowstatic.com/research/public...,"Zillow Home Value Index (ZHVI): A smoothed, se...",,
5,zhvi_4_homes_by_zip,https://files.zillowstatic.com/research/public...,"Zillow Home Value Index (ZHVI): A smoothed, se...",,
6,zhvi_3_homes_by_zip,https://files.zillowstatic.com/research/public...,"Zillow Home Value Index (ZHVI): A smoothed, se...",,
7,zhvi_2_homes_by_zip,https://files.zillowstatic.com/research/public...,"Zillow Home Value Index (ZHVI): A smoothed, se...",,
8,zhvi_1_homes_by_zip,https://files.zillowstatic.com/research/public...,"Zillow Home Value Index (ZHVI): A smoothed, se...",,
9,zhvi_single_family_homes_by_county,https://files.zillowstatic.com/research/public...,"Zillow Home Value Index (ZHVI): A smoothed, se...",,


In [74]:
# Fill in illowdf dataframe's 'data' clumn with indvidual dataframes for each individual data source
for i, row in zillow_df.iterrows():
    
    url=row['url']
    var_name=zillow_df.loc[i]['stat_name']
    print(f"PROCESSNG................{var_name}")

    #   start filling in 'data' column with df output by pull_zllw_data f-n
    zillow_df.iloc[i]['data']=pull_zillow_data(url, state, county, city, var_name)

    #   optionally create indemendent standalone data frames   
    vars()[zillow_df.loc[i]['stat_name']+ "_df"]=zillow_df.iloc[i]['data']
    
    # Label data sources by entity type (zip code, city, county, country etc)    
    if 'County' in zillow_df.iloc[i]['data']['RegionType'].unique() :
        zillow_df.iloc[i]['type']='City'
    elif 'Msa' in zillow_df.iloc[i]['data']['RegionType'].unique() :
        zillow_df.iloc[i]['type']='City'
    elif 'Zip' in zillow_df.iloc[i]['data']['RegionType'].unique() :
        zillow_df.iloc[i]['type']='City'

PROCESSNG................list_price_data
PROCESSNG................sale_data
PROCESSNG................inventory_d2p_data
PROCESSNG................inventory_4s_data
PROCESSNG................zhvi_5plus_homes_by_zip
PROCESSNG................zhvi_4_homes_by_zip
PROCESSNG................zhvi_3_homes_by_zip
PROCESSNG................zhvi_2_homes_by_zip
PROCESSNG................zhvi_1_homes_by_zip
PROCESSNG................zhvi_single_family_homes_by_county
PROCESSNG................zhvi_condo_coop_homes_by_county
PROCESSNG................zhvi_5plus_homes_by_county
PROCESSNG................zhvi_4_homes_by_county
PROCESSNG................zhvi_3_homes_by_county
PROCESSNG................zhvi_2_homes_by_county
PROCESSNG................zhvi_1_homes_by_county


In [47]:
# zillow_df['type']=""
zillow_df

,stat_name,url,data,type
0,list_price_data,https://files.zillowstatic.com/research/public...,RegionName Regio...,
1,sale_data,https://files.zillowstatic.com/research/public...,RegionName Regi...,
2,inventory_d2p_data,http://files.zillowstatic.com/research/public_...,RegionName Regio...,
3,inventory_4s_data,https://files.zillowstatic.com/research/public...,RegionName Regio...,
4,zhvi_5plus_homes_by_zip,https://files.zillowstatic.com/research/public...,RegionName RegionType StateName State...,
5,zhvi_4_homes_by_zip,https://files.zillowstatic.com/research/public...,RegionName RegionType StateName State...,
6,zhvi_3_homes_by_zip,https://files.zillowstatic.com/research/public...,RegionName RegionType StateName State...,
7,zhvi_2_homes_by_zip,https://files.zillowstatic.com/research/public...,RegionName RegionType StateName State...,
8,zhvi_1_homes_by_zip,https://files.zillowstatic.com/research/public...,RegionName RegionType StateName State...,
9,zhvi_single_family_homes_by_county,https://files.zillowstatic.com/research/public...,RegionName RegionType StateNam...,


In [ ]:
chosen_cities=["Houston", "San Francisco", "New York", "Chicago"]

for city in chosen_cities:
     